In [161]:
from flask import Flask, jsonify
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from dateutil import relativedelta
import os
import time


month = int(input('Enter the month: '))
year = int(input("Enter the year: "))
response = requests.post('https://mahaepos.gov.in/FPS_Trans_Details.jsp', data = {'dist_code':2518, 'fps_id':251832900166, 'month': month, 'year': year})
time1 = time.time()
soup = BeautifulSoup(response.text, 'html.parser')
headers = []
thElements = [x.text.strip() for x in soup.find_all('th')]
for i in thElements[1:thElements.index('Total')]:
  if i != 'Qty in Kgs':
    headers.append(i)
headers.remove('Amount(Rs.)')
headers.remove('Portability')
headers.remove('Auth Trans Time')
headers.append('Amount(Rs.)')
headers.append('Portability')
headers.append('Auth Trans Time')

data = pd.read_html(response.text)
df = pd.DataFrame(data[0].values.tolist(), columns=headers)

collist = []
for col in df.columns:
  if df[col].dtype == 'float64' and df[col].iloc[-1]> 0 :
    collist.append(col)
  
df.set_index('Date' , inplace=True)
df.drop(index = 'Total' , inplace= True )
df.index = df.index.map(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%S'))

# dailysales = df.resample('D')['Wheat(Kgs)' ,'Rice(Kgs)' , 'Wheat-PMGKAY(Kgs)' , 'Rice-PMGKAY(Kgs)' ].sum()
dailysales = df.resample('D')[collist].sum()
dailysales = dailysales.merge(df.resample('D')['Wheat(Kgs)'].count().rename('Sale Count') , on='Date')
monthstring = dailysales.index[0]
# print(monthstring)
dailysales.index = dailysales.index.map(str).map(lambda x : x[0:11])
dailysales.loc[-1] = dailysales.sum()
#dailysales.rename(columns = {'Wheat(Kgs)':'wheat', 'Rice(Kgs)':'rice', 'Wheat-PMGKAY(Kgs)':'wheatpmgkay', 'Rice-PMGKAY(Kgs)': 'ricepmgkay', 'Sale Count':'salecount'}, inplace = True)
dailysales.rename(index = {-1 :'Total'} , inplace = True)
# dailysales.reset_index(inplace=True)

Enter the month: 5
Enter the year: 2023


In [162]:
pathlist = []
for dir , subdir , files in os.walk(r'C:\Users\ysaur\OneDrive\Ration\Daily sales\ds_excel/'):
  for file in files:
    pathlist.append(os.path.join(dir,file))

In [172]:
sheetname = 'remaining_cards'
remaining_cards = pd.DataFrame()
for location in pathlist:
  remdf  = pd.read_excel(location , sheet_name = sheetname , engine = "openpyxl")
  remaining_cards = pd.concat([remaining_cards, remdf] , ignore_index = True)

In [173]:
datestring = datetime.strptime('2022-08-01' , '%Y-%m-%d')
total_months = 12*relativedelta.relativedelta(datetime.now(), datestring).years + relativedelta.relativedelta(datetime.now(), datestring).months

In [193]:
probability = remaining_cards.groupby('SRC No').count()['Units']
probability = round(probability/total_months, 4)
probability = 100 - (probability*100)
probability[probability < 0] = 0
probability.name = 'Percentage'

KeyError: 'SRC No'

In [176]:
probability

SRC No
272025608995    88.89
272025609037    66.67
272025609053    33.33
272025609054    77.78
272025609058    88.89
272025609060    22.22
272025609101    33.33
272025609104    77.78
272025609107    88.89
272025609109    77.78
272025609110    88.89
272025609111    88.89
272025609112    88.89
272025609113    88.89
272025609122    11.11
272025609125    88.89
272025616740    88.89
272025616817    88.89
272025617713    77.78
272025617716    88.89
272025625560    88.89
272025626676    77.78
272025627175    55.56
272025627179    55.56
272025627180    77.78
272025627182    66.67
272025627183    66.67
272025627185    88.89
272025627187    88.89
272025627192    55.56
272025627193    55.56
272025627197    88.89
272025627202    77.78
272025627203    88.89
272025627205    88.89
272025627209    88.89
272025627210    88.89
272025627220    88.89
272025627221    88.89
272025627224    88.89
272025627226    11.11
272025627231    88.89
272025627232     0.00
272025627234    66.67
272025627236    22.22
272

In [177]:
cardBase = pd.read_excel(r'C:\Users\ysaur\OneDrive\Ration\Daily sales\dataset/remaningcards.xlsx' )
yetTOcome = cardBase[cardBase['SRC No'].isin(df['SRC No'].astype('float'))==False].sort_values(by = 'REF').fillna(0).astype('int64').reset_index(drop = True)
yetTOcome = yetTOcome.merge(probability, on = 'SRC No' , how = 'left')

In [178]:
yetTOcome

,SRC No,REF,Units,Percentage
0,272025647517,6,5,77.78
1,272025647534,15,5,88.89
2,272025647539,16,2,0.00
3,272025627183,18,5,66.67
4,272025647544,19,6,88.89
5,272025647563,29,5,88.89
6,272025627838,31,6,88.89
7,272026502234,34,4,44.44
8,272026590163,36,6,88.89
9,272025647574,51,5,66.67


In [186]:
import requests
import concurrent.futures
from bs4 import BeautifulSoup

def make_request(i, month, year):
    url = 'http://mahaepos.gov.in/SRC_Trans_Details.jsp'
    data = {'src_no': i, 'month': month, 'year': year}
    response = requests.post(url, data=data)
    soup = BeautifulSoup(response.text, 'html.parser')
    tableElements = soup.find_all('table')
    if len(tableElements) > 2:
        trElements = tableElements[-1].findChildren('tr')
        tdElements = trElements[-1].findChildren('td')
        if tdElements[2].text != '251832900166':
            return i, 'Ported'
        else:
            return i, 'Taken'
    else:
        return i, 'Pending'

def main():
    month = datetime.now().month
    year = datetime.now().year
  # Your yetTOcome data
    status = []
    with concurrent.futures.ThreadPoolExecutor(max_workers= 216) as executor:
        # Submit the tasks to the executor
        futures = [executor.submit(make_request, i, month, year) for i in yetTOcome['SRC No']]

        # Retrieve the results as they become available
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            status.append(result)
            print(result[0])
        print('ended')
    return status
    # Process the status list further if needed
